# Simulation: Dependence on Break Locations with Weighted Loss

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time
from sklearn.model_selection import train_test_split
import time

In [ ]:
from copy import deepcopy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from datetime import datetime

Set parameters:

In [ ]:
path = 'C:/Users/Meier/Dropbox (Institut für Statistik)/Structural Breaks + DL/Figures/Results/' 

In [ ]:
model_name = 'RNN' # 'RNN', 'LSTM', 'GRU'

In [ ]:
reps = 1000           # number of repetitions
sim_length = 500      # length of simulated sample

In [ ]:
test_size = 0.1           # proportion of test set
lags = 1                  # number of lags as features

In [ ]:
weight_type = 'tukey-hanning' # 'exponential', 'rayleigh', 'bartlett', 'parzen' , 'tukey-hanning'
if weight_type == 'bartlett' or weight_type == 'parzen' or weight_type == 'tukey-hanning':
    fac = 1 # {1, 0.95, 0.9}
    alpha = (sim_length*(1-test_size))**fac   # kernels: {T, T^0.95, T^0.9}
else:
    alpha = 0.005 # exp: {0.005, 0.01, 0.02}, ray: {2*10**(-5), 5*10**(-5), 1*10**(-4)}

In [ ]:
batch_size = 128
n_epochs = 50

In [ ]:
# set parameters
input_dim = lags                  # number of lagged features in X
hidden_dim = 10                   # number of hidden nodes per layer
layer_dim = 1                     # number of layers
output_dim = 1                    # output dimension (1 for univariate output)
dropout = 0                       # dropout proportion (only before the last sequential layer)
learning_rate = 1e-3              # learning rate for Adam optimizer
weight_decay = 1e-6               # weight decay for Adam optimizer

# save model parameters in dict
model_params = {'input_dim': input_dim, 'hidden_dim' : hidden_dim,'layer_dim' : layer_dim, 'output_dim' : output_dim, 'dropout_prob' : dropout}        

Models:

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        self.rnn.flatten_parameters() # ------------------------------------------------------------------
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out
    
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        self.lstm.flatten_parameters() # ------------------------------------------------------------------
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        self.gru.flatten_parameters() # ------------------------------------------------------------------
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
def get_model(model, model_params):
    models = {
        "rnn": RNNModel,
        "lstm": LSTMModel,
        "gru": GRUModel,
    }
    return models.get(model.lower())(**model_params)

In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
    
    def train_step(self, x, y, weight):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat, weight)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()
    
    def train(self, train_loader, weight_type, alpha, batch_size=64, n_epochs=50, n_features=1):
        
        # train on GPU
        device = torch.device('cuda')

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            obs_total = len(train_loader.dataset)  # number of samples
            step = 1
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([len(y_batch), -1, n_features]).to(device)
                y_batch = y_batch.view([len(y_batch), -1]).to(device)
                t = np.arange(step,step+len(y_batch)) # array of periods of this batch
                if weight_type == 'exponential':
                    weight = np.exp(-alpha*(obs_total-t))  # exponential weight
                if weight_type == 'rayleigh':
                    weight = np.exp(-(1/2)*alpha*(obs_total-t)**2)  # Rayleigh weight
                if weight_type == 'bartlett':  # Bartlett weight
                    weight = np.zeros([t.shape[0]])
                    for i in range(t.shape[0]):
                        if ((obs_total-t[i])/alpha)<=1:
                            weight[i] = 1-(obs_total-t[i])/alpha
                        else:
                            weight[i] = 0
                if weight_type == 'parzen':  # Parzen weight
                    weight = np.zeros([t.shape[0]])
                    for i in range(t.shape[0]):
                        if ((obs_total-t[i])/alpha)>=0 and ((obs_total-t[i])/alpha)<=0.5:
                            weight[i] = (1-6*((obs_total-t[i])/alpha)**2+6*((obs_total-t[i])/alpha)**3) 
                        elif ((obs_total-t[i])/alpha)>0.5 and ((obs_total-t[i])/alpha)<=1:
                            weight[i] = 2*(1-((obs_total-t[i])/alpha))**3 
                        else:
                            weight[i] = 0
                if weight_type == 'tukey-hanning':  # Tukey-Hanning weight
                    weight = np.zeros([t.shape[0]])
                    for i in range(t.shape[0]):
                        if ((obs_total-t[i])/alpha)<=1:
                            weight[i] = (1+np.cos(np.pi*((obs_total-t[i])/alpha)))/2
                        else:
                            weight[i] = 0
                b_loss = self.train_step(x_batch, y_batch, torch.tensor(weight).to(device))
                batch_losses.append(b_loss)
                step += len(y_batch) ###########################################
            training_loss = np.sum(batch_losses)
            self.train_losses.append(training_loss)

        return self.model

    def evaluate(self, best_model, test_loader, batch_size=1, n_features=1):
        # evaluate on GPU
        device = torch.device('cuda')
        model = deepcopy(best_model)
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.view([batch_size, -1]).to(device)
                model.eval()
                yhat = model(x_test)
                predictions.append(yhat.to(device).detach().cpu().numpy())
                values.append(y_test.to(device).detach().cpu().numpy())

        return predictions, values
    
    def plot_losses(self):
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [ ]:
def format_predictions_dl(predictions, values):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={"value": vals, "prediction": preds})
    df_result = df_result.sort_index()
    return df_result

def calculate_metrics(df):
    return {'rmse' : mean_squared_error(df.value, df.prediction)**0.5,
            'mae' : mean_absolute_error(df.value, df.prediction),
            'mape': mean_absolute_percentage_error(df.value, df.prediction),
            'r2' : r2_score(df.value, df.prediction)}

def time_format(seconds: int):
    if seconds is not None:
        seconds = int(seconds)
        d = seconds // (3600 * 24)
        h = seconds // 3600 % 24
        m = seconds % 3600 // 60
        s = seconds % 3600 % 60
        if d > 0:
            return '{:02d}D {:02d}H {:02d}m {:02d}s'.format(d, h, m, s)
        elif h > 0:
            return '{:02d}H {:02d}m {:02d}s'.format(h, m, s)
        elif m > 0:
            return '{:02d}m {:02d}s'.format(m, s)
        elif s > 0:
            return '{:02d}s'.format(s)
    return '-'

In [ ]:
class WeightedLoss(nn.Module):
    def __init__(self):
        super(WeightedLoss, self).__init__()
 
    def forward(self, inputs, targets, weight=1):  
        
        # flatten input and target tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        # get weighted squared differences
        weighted_sqrd_diff = weight*((inputs - targets) ** 2)
        
        return weighted_sqrd_diff.sum()

Data:

In [ ]:
# loop over break locations
tau = np.arange(0,1,0.05)
data = []
for j in range(len(tau)):
        
    # simulate constant data (with mean break from 0.5 to 0)
    if j == 0:
        data_temp = 0.5*np.ones(int(sim_length)+lags)
    else:
        data_temp = np.concatenate((np.zeros(int(np.round((1-test_size)*sim_length*tau[j],0)+lags)),0.5*np.ones(int(np.round((1-test_size)*sim_length*(1-tau[j])+test_size*sim_length,0)))),axis=0)   

    # split into training and test data
    X_temp = data_temp[:-1]
    y_temp = data_temp[1:]
    data.append(train_test_split(X_temp, y_temp, test_size=test_size, shuffle=False)) # X_train, X_test, y_train, y_test for every j in tau 

Training:

In [ ]:
# start timer
timer_start = time.time()
print('Simulation start: %s' %time.ctime(int(timer_start)))

# run specified number of repetitions
results = []
preds = []
for i in range(reps):
    
    # print repetition
    print('Repetition: ',i+1)
    
    # train on GPU
    device = torch.device('cuda')
    
    # loop over all break locations
    df_sim = pd.DataFrame()
    for j in range(len(tau)):
        
        # extract data
        X_train = data[j][0]
        X_test = data[j][1]
        y_train = data[j][2]
        y_test = data[j][3]
        
        # convert data to tensors
        train_features = torch.Tensor(X_train)
        train_targets = torch.Tensor(y_train)
        test_features = torch.Tensor(X_test)
        test_targets = torch.Tensor(y_test)
        
        # build tensor dataset
        train = TensorDataset(train_features, train_targets)
        test = TensorDataset(test_features, test_targets)

        # get batched data
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=False) 
        test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)
        
        # initialise model
        torch.manual_seed(i)
        model = get_model(model_name, model_params).to(device)
        
        # create loss function and optimizer
        loss_fn = WeightedLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
        
        # train the model
        model_out = opt.train(train_loader, weight_type=weight_type, alpha=alpha, batch_size=batch_size, n_epochs=n_epochs, n_features=1)
            
        # evaluate on test set
        predictions, values = opt.evaluate(model_out, test_loader_one, batch_size=1, n_features=1)
        if j == 0:
            pred_bl = np.expand_dims(np.squeeze(predictions),axis=0)
        else:
            pred_bl = np.concatenate((pred_bl,np.expand_dims(np.squeeze(predictions),axis=0)),axis=0)
        df_result = format_predictions_dl(predictions, values)
        result_metrics = calculate_metrics(df_result)
        
        # append metrics on test set
        df_metrics = pd.DataFrame(np.expand_dims((result_metrics['rmse'],result_metrics['mae'],result_metrics['mape'],result_metrics['r2'],),axis=0),columns=['mse','mae','mape','r2'])
        df_sim = pd.concat([df_sim,df_metrics],axis=0, ignore_index=True)
    
    # save results of every repetition
    results.append(df_sim) 
    preds.append(pred_bl)
    
    if (i < 10) | (i % 50 == 0):
        print('Elapsed: %s' %time_format(time.time() - timer_start))

Evaluate and save results:

In [ ]:
arr_results = np.asarray(results) # dims: no. of reps x no. of break locations x no. of metrics
arr_preds = np.asarray(preds) # dims: no. of reps x no. of break locations x no. of metrics

In [ ]:
# metrics
arr_mean = np.mean(arr_results,axis=0)
arr_std = np.std(arr_results,axis=0)
arr_min = np.min(arr_results,axis=0)
arr_max = np.max(arr_results,axis=0)
arr_median = np.median(arr_results,axis=0)

# predictions
arr_mean_pred = np.mean(arr_preds,axis=0)
arr_std_pred = np.std(arr_preds,axis=0)
arr_min_pred = np.min(arr_preds,axis=0)
arr_max_pred = np.max(arr_preds,axis=0)
arr_median_pred = np.median(arr_preds,axis=0)

In [ ]:
if weight_type == 'bartlett' or weight_type == 'parzen' or weight_type == 'tukey-hanning':
    np.savez(path+'Plot_'+model_name+'_'+weight_type+'_T'+str(fac)+'_results.npz',mean=arr_mean,std=arr_std,minimum=arr_min,maximum=arr_max,median=arr_median)
    np.savez(path+'Plot_'+model_name+'_'+weight_type+'_T'+str(fac)+'_preds.npz',mean=arr_mean_pred,std=arr_std_pred,minimum=arr_min_pred,maximum=arr_max_pred,median=arr_median_pred)
else:
    np.savez(path+'Plot_'+model_name+'_'+weight_type+'_'+str(alpha)+'_results.npz',mean=arr_mean,std=arr_std,minimum=arr_min,maximum=arr_max,median=arr_median)
    np.savez(path+'Plot_'+model_name+'_'+weight_type+'_'+str(alpha)+'_preds.npz',mean=arr_mean_pred,std=arr_std_pred,minimum=arr_min_pred,maximum=arr_max_pred,median=arr_median_pred)